In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/emaillstm/pytorch/default/1/modelCombined1.pth
/kaggle/input/email-classification-dataset/glove.6B.200d.txt
/kaggle/input/email-classification-dataset/glove.6B.50d.txt
/kaggle/input/email-classification-dataset/glove.6B.300d.txt
/kaggle/input/email-classification-dataset/combined_data.csv
/kaggle/input/email-classification-dataset/glove.6B.100d.txt


In [2]:
# import pandas as pd
# import re

# def remove_punctuation(text):
#     return re.sub(r'[^\w\s]', '', str(text).strip())

# def remove_extra_spaces(text):
#     return re.sub(' +', ' ', text)

# df = pd.read_csv('/kaggle/input/spamnotspamkaggle/spam_or_not_spam.csv')
# # df['Text'] = df['Message'] if not NaN else df['Subject']

# # df['Text'] = df['Message'].fillna(df['Subject'])

# # df = df.drop(columns=['Message', 'Subject', 'Message ID', 'Date'], axis=1)

# df["Text"] = df["email"].apply(remove_punctuation)
# df["Text"] = df["Text"].str.replace('\n', ' ')
# df["Text"] = df["Text"].apply(remove_extra_spaces)
# df["Text"] = df["Text"].str.strip()
# df["Text"] = df["Text"].str.lower()
# df["Text"] = df["Text"].str.split(' ')

# df = df.drop(columns=['email'], axis=1)

# df['Spam'] = df['label']
# # df["Spam/Ham"] = df["Spam/Ham"].map({'spam': 1, 'ham': 0})

# df

In [3]:
import pandas as pd
import re

df = pd.read_csv('/kaggle/input/email-classification-dataset/combined_data.csv')

def remove_punctuation(text):
    return re.sub(r'[^\w\s]', '', str(text).strip())

def remove_extra_spaces(text):
    return re.sub(' +', ' ', text)

# df = pd.read_csv('Data/spam_or_not_spam/spam_or_not_spam.csv')
# df['Text'] = df['Message'] if not NaN else df['Subject']

# df['Text'] = df['Message'].fillna(df['Subject'])

# df = df.drop(columns=['Message', 'Subject', 'Message ID', 'Date'], axis=1)

df["Text"] = df["text"].apply(remove_punctuation)
df["Text"] = df["Text"].str.replace('\n', ' ')
df["Text"] = df["Text"].apply(remove_extra_spaces)
df["Text"] = df["Text"].str.strip()
df["Text"] = df["Text"].str.lower()
df["Text"] = df["Text"].str.split(' ')

df['Spam'] = df['label']

df = df.drop(columns=['text', 'label'], axis=1)
# df["Spam/Ham"] = df["Spam/Ham"].map({'spam': 1, 'ham': 0})

df

,Text,Spam
0,"[ounce, feather, bowl, hummingbird, opec, mome...",1
1,"[wulvob, get, your, medircations, online, qnb,...",1
2,"[computer, connection, from, cnn, com, wednesd...",0
3,"[university, degree, obtain, a, prosperous, fu...",1
4,"[thanks, for, all, your, answers, guys, i, kno...",0
...,...,...
83443,"[hi, given, a, date, how, do, i, get, the, las...",0
83444,"[now, you, can, order, software, on, cd, or, d...",1
83445,"[dear, valued, member, canadianpharmacy, provi...",1
83446,"[subscribe, change, profile, contact, us, long...",0


In [4]:
import numpy as np
import torch
import torch.nn.utils.rnn as rnn_utils

# Define a function to load GloVe vectors
def load_glove_vectors(filepath):
    word_to_vec = {}
    with open(filepath, 'r', encoding='utf-8') as file:
        for line in file:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            word_to_vec[word] = vector

    word_to_vec['<eol>'] = np.zeros((100,))
    word_to_vec['<unk>'] = np.zeros((100,))
    return word_to_vec

# Load the vectors
glove_vectors = load_glove_vectors('/kaggle/input/email-classification-dataset/glove.6B.100d.txt')

# glove_vectors["don't"]


In [5]:
X = df['Text'].values
Y = df['Spam'].values

X_vec = []
# Y_vec = np.zeros((2, len(Y)))
Y_vec = np.array(Y)

for i in range(len(X)):
  vec = np.zeros((100, len(X[i])))
  for j in range(len(X[i])):
    if X[i][j] in glove_vectors:
      vec[:, j] = glove_vectors[X[i][j]]
    else:
      vec[:, j] = glove_vectors['<unk>']

  X_vec.append(torch.tensor(vec.reshape(-1, 100), dtype=torch.float32))

Y_vec = torch.tensor(Y_vec, dtype=torch.float32)
#   Y_vec[Y[i] - 1, i] = 1

# random = np.random.permutation(len(X_vec))
# theshold = int(0.8*len(random))
# X_train = [X_vec[i].reshape(-1,100) for i in random[0:theshold]]
# Y_train = Y_vec[random[0:theshold]]

# X_test = [X_vec[i].reshape(-1,100) for i in random[theshold:]]
# Y_test = Y_vec[random[theshold:]]

# Y_train = Y_train.reshape(-1, 1)
# Y_test = Y_test.reshape(-1, 1)

X_vec = X_vec
Y_vec = Y_vec.reshape(-1, 1)

In [6]:
# # print("Lengths:", lengths)
# print("Padded Sequences Shape:", X.shape)

# print("X_padded Device:", X.device)
# print("Lengths Device:", lengths.device)


In [7]:
Y_vec[1]

tensor([1.])

In [8]:
X_vec[0].shape

torch.Size([20, 100])

In [9]:
# X = torch.tensor(X, dtype=torch.float32)
# X = [torch.tensor(x, dtype=torch.float32) for x in X_vec]

X = X_vec
Y = Y_vec

print(X[0].size())

# lengths = [seq.size(0) for seq in X]

# print(max(lengths))

# X = rnn_utils.pad_sequence(X, batch_first=True, padding_value=0)
# Y = torch.tensor(Y_vec, dtype=torch.float32)

batch_size = 16

# dataset = torch.utils.data.TensorDataset(X, Y)
dataset = [(X[i], Y[i]) for i in range(len(X))]
dataset.sort(key = lambda x: x[0].size(0))

X, Y = zip(*dataset)

dataset = [(rnn_utils.pad_sequence(X[i: i + batch_size],
                                  batch_first = True,
                                  padding_value=0),
               torch.stack(Y[i: i + batch_size])) for i in range(0, len(dataset), batch_size)]

print(len(dataset))
# print(dataset[0][0].size())
# print(dataset[0][1].size())

train_size = int(0.99 * len(dataset))
# test_size = len(dataset) - train_size

random = np.random.permutation(len(dataset))

train_dataset = [dataset[i] for i in random[0:train_size]]
test_dataset = [dataset[i] for i in random[train_size:]]

# train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# print(train_dataset)

# train_loader = torch.utils.data.DataLoader(train_dataset,
#                                            batch_size = batch_size,
#                                            shuffle = True)

# test_loader = torch.utils.data.DataLoader(test_dataset,
#                                            batch_size = batch_size,
#                                            shuffle = False)

# TrainLengths = torch.tensor([[x[0].size(0)] * batch_size for x in train_loader], dtype=torch.int64)
# TestLengths = torch.tensor([[x[0].size(0)]* batch_size for x in test_loader], dtype=torch.int64)

# print(max(TrainLengths[:,0]))

torch.Size([20, 100])
5216


In [10]:
print(dataset[1000][0].size())
print(dataset[1000][1].size())

torch.Size([16, 55, 100])
torch.Size([16, 1])


In [11]:
def calLenghts(data):
    
    device = data.device
    batch_size = data.size(0)
    sequence_length = data.size(1)
    
    lengths = torch.ones(batch_size, dtype = torch.int64)
    
    for i in range(batch_size):
        
        seq = data[i]
        
        non_zero_index = (seq != 0).any(dim=1).nonzero(as_tuple=True)[0]
        
        if len(non_zero_index) > 0:
            lengths[i] = non_zero_index[-1] + 1
        
    lengths = lengths.to('cpu')
    return lengths
    

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch'cpu'

# lengths = torch.tensor(lengths, dtype=torch.long)
# lengths = lengths.cpu()
# lengths = lengths.to(torch.int64)

# TrainLengths = TrainLengths.to('cpu')
# TestLengths = TestLengths.to('cpu')

# print("TestLenghts", TestLengths.shape)
# print("TrainLengths", TrainLengths.shape)

In [13]:
import torch.nn as nn

input_shape = 100
hidden_shape = 200
output_shape = 1
num_layers = 4
nepochs = 100
dropoput = 0.1
learning_rate = 0.0001

class LSTM(nn.Module):
  def __init__(self):
    super(LSTM, self).__init__()

    # self.L1 = nn.Linear(input_shape, 15)
    # self.L2 = nn.Linear(15, 20)
    # self.L3 = nn.Linear(20, 20)
    self.LSTM = nn.LSTM(input_shape, hidden_shape, num_layers = num_layers, batch_first=True, dropout=dropoput)

    # batch size, sequence length, no. of feature (100)

    self.out1 = nn.Linear(hidden_shape, 100)
    self.out2 = nn.Linear(100, 10)
    self.out3 = nn.Linear(10, output_shape)
    
    self.dropout = nn.Dropout(dropoput)

    self.relu = nn.ReLU()

  def forward(self, x, lengths):
    packed_input = rnn_utils.pack_padded_sequence(x, lengths, batch_first=True, enforce_sorted=False)
    packed_output, _ = self.LSTM(packed_input)
    output, _ = rnn_utils.pad_packed_sequence(packed_output, batch_first=True)
    
#     print("output", output.size())

    x = output[:, -1, :]
    
#     print("output", output.size())

    x = self.relu(x)
    x = self.dropout(self.relu(self.out1(x)))
    x = self.dropout(self.relu(self.out2(x)))
    x = self.out3(x)

    return x

model = LSTM().to(device)

# model.load_state_dict(torch.load('/kaggle/input/emaillstm/pytorch/default/1/modelCombined1.pth'))

criterion = nn.BCEWithLogitsLoss()
# criterion = nn.MSELoss()
# optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

for epoch in range(nepochs):
#   l = 0
  running_loss = 0.0
  for data, label in train_dataset:
#     print(TrainLengths[l])
    data = data.to(device)
#     print(calLenghts(data))
    output = model(data, calLenghts(data))
    label = label.to(device)
    loss = criterion(output, label)
    
    running_loss += loss.item()
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
#     l += 1

  print(f'Epoch [{epoch+1}/{nepochs}], Loss: {running_loss / len(train_dataset):.4f}')

Epoch [1/100], Loss: 0.6292
Epoch [2/100], Loss: 0.5309
Epoch [3/100], Loss: 0.4560
Epoch [4/100], Loss: 0.3891
Epoch [5/100], Loss: 0.3427
Epoch [6/100], Loss: 0.3091
Epoch [7/100], Loss: 0.2832
Epoch [8/100], Loss: 0.2631
Epoch [9/100], Loss: 0.2465
Epoch [10/100], Loss: 0.2313
Epoch [11/100], Loss: 0.2178
Epoch [12/100], Loss: 0.2056
Epoch [13/100], Loss: 0.1950
Epoch [14/100], Loss: 0.1867
Epoch [15/100], Loss: 0.1768
Epoch [16/100], Loss: 0.1692
Epoch [17/100], Loss: 0.1608
Epoch [18/100], Loss: 0.1566
Epoch [19/100], Loss: 0.1479
Epoch [20/100], Loss: 0.1432
Epoch [21/100], Loss: 0.1374
Epoch [22/100], Loss: 0.1340
Epoch [23/100], Loss: 0.1276
Epoch [24/100], Loss: 0.1239
Epoch [25/100], Loss: 0.1211
Epoch [26/100], Loss: 0.1191
Epoch [27/100], Loss: 0.1147
Epoch [28/100], Loss: 0.1136
Epoch [29/100], Loss: 0.1098
Epoch [30/100], Loss: 0.1085
Epoch [31/100], Loss: 0.1058
Epoch [32/100], Loss: 0.1030
Epoch [33/100], Loss: 0.1016
Epoch [34/100], Loss: 0.1006
Epoch [35/100], Loss: 0

In [14]:
model.eval()

with torch.no_grad():
    ncorrected = 0
    # n = len(test_loader.dataset)
    n = 0

    for data, label in train_dataset:
        data = data.to(device)
        label = label.reshape(-1, 1).to(device)
        output = model(data, calLenghts(data))
        output = torch.sigmoid(output)
        # Apply a threshold to the output to get the predicted class
        predicted = (output >= 0.5).float()  # Convert probabilities to 0 or 1

        ncorrected += (predicted == label).sum().item()
        n += label.size(0)

    acc = ncorrected / n

    print(f'Train Accuracy of the network: {100 * acc:.2f} %')


Train Accuracy of the network: 95.64 %


In [15]:
with torch.no_grad():
    ncorrected = 0
    # n = len(test_loader.dataset)
    n = 0

    for data, label in test_dataset:
        data = data.to(device)
        label = label.reshape(-1, 1).to(device)
        output = model(data, calLenghts(data))
        output = torch.sigmoid(output)
        # Apply a threshold to the output to get the predicted class
        predicted = (output >= 0.5).float()  # Convert probabilities to 0 or 1

        ncorrected += (predicted == label).sum().item()
        n += label.size(0)

    acc = ncorrected / n

    print(f'Test Accuracy of the network: {100 * acc:.2f} %')


Test Accuracy of the network: 91.51 %


In [16]:
torch.save(model.state_dict(), "modelCombined1_nlayer4_hidden_200batch16dropout02.pth")